In [1]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle, params 
using Base.Iterators: repeated, partition
using Printf, BSON
using CSV
using DataFrames
using Tables
using Random
using Plots
using Turing, ReverseDiff

Turing.setprogress!(false);
Turing.setadbackend(:reversediff)

┌ Info: [Turing]: progress logging is disabled globally
└ @ Turing C:\Users\XudongT\.julia\packages\Turing\S4Y4B\src\Turing.jl:22
┌ Info: [AdvancedVI]: global PROGRESS is set as false
└ @ AdvancedVI C:\Users\XudongT\.julia\packages\AdvancedVI\W2zsz\src\AdvancedVI.jl:15


:reversediff

In [2]:
BSON.@load "AENN.bson" model

In [3]:
original_dat = CSV.read("../processed-data/otu-yield-per-plant.csv", header=true, DataFrame)
generated_dat = CSV.read("../processed-data/generated-data.csv", header=false, DataFrame)

original = Matrix(original_dat)
generated = Matrix(generated_dat)
full_data = vcat(original, generated)

otu = full_data[:,2:60]
label = full_data[:,62];

In [4]:
code = Array{Float32}(undef, size(otu)[1], 5)
for i in 1:size(otu)[1]
    code[i,:] .= model[1](otu[i,:])
end

In [19]:
code_vec = Array([[code[i,1];code[i,2];code[i,3];code[i,4];code[i,5]] for i in 1:size(otu)[1]])
label = convert(Array{Float64,1}, label);

In [20]:
# construct the nn
nn_initial = Chain(Dense(5, 3, tanh), Dense(3, 1, σ))

Chain(
  Dense(5 => 3, tanh),                  # 18 parameters
  Dense(3 => 1, σ),                     # 4 parameters
)                   # Total: 4 arrays, 22 parameters, 344 bytes.

In [22]:
parameters_initial, reconstruct = Flux.destructure(nn_initial);
length(parameters_initial)

22

In [23]:
# now create a parameter variable that has IID Gaussian Distribution
alpha = 0.09
# variance for the gaussian prior
sig = sqrt(1.0 / alpha)

# Specify the probabilistic model.
@model function bayes_nn(xs, ts, nparameters, reconstruct)
    # Create the weight and bias vector.
    parameters ~ MvNormal(zeros(nparameters), sig .* ones(nparameters))

    # Construct NN from parameters
    nn = reconstruct(parameters)
    # Forward NN to make predictions
    preds = nn(xs)

    # Observe each prediction.
    for i in 1:length(ts)
        ts[i] ~ Bernoulli(preds[i])
    end
end

bayes_nn (generic function with 2 methods)

In [24]:
# Perform inference.
N = 5000
ch = sample(
    bayes_nn(hcat(code_vec...), label, length(parameters_initial), reconstruct), HMC(0.05, 4), N
)

Chains MCMC chain (5000×31×1 Array{Float64, 3}):

Iterations        = 1:1:5000
Number of chains  = 1
Samples per chain = 5000
Wall duration     = 320.67 seconds
Compute duration  = 320.67 seconds
parameters        = parameters[1], parameters[2], parameters[3], parameters[4], parameters[5], parameters[6], parameters[7], parameters[8], parameters[9], parameters[10], parameters[11], parameters[12], parameters[13], parameters[14], parameters[15], parameters[16], parameters[17], parameters[18], parameters[19], parameters[20], parameters[21], parameters[22]
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, step_size, nom_step_size

Summary Statistics
      parameters      mean       std   naive_se      mcse       ess      rhat  ⋯
          Symbol   Float64   Float64    Float64   Float64   Float64   Float64  ⋯

   parameters[1]   -3.9434    2.4684     0.0349    0.2889   11.7280    1.3585  ⋯
   parameters[2]    4.6076    4.3

In [26]:
theta = MCMCChains.group(ch, :parameters).value;

In [28]:
nn_forward(x, theta) = reconstruct(theta)(x)
function nn_predict(x, theta, num)
    return mean([nn_forward(x, theta[i, :])[1] for i in 1:10:num])
end;